# Data Metrics
### Imports

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib
import tqdm.auto as tqdm

import os
from PIL import Image
from sklearn.metrics import accuracy_score
import torchvision
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc

import torchvision.models as models

plt.style.use('seaborn')

import DiagnosisFunctions.tools as tools

import albumentations as A
import torchvision.transforms.functional as TF

### Read files

In [2]:
ground_truths = pd.read_csv('GT.csv')
predictions = pd.read_csv('pred.csv')

In [5]:
pred_class = tools.classify_diagnosis_predictions(predictions)

In [8]:
print(tools.compute_diagnosis_scores(ground_truths, pred_class))

0.1781317299385846


## Classify predictions

In [7]:
pred_class = tools.classify_probability_predictions(predictions)
pred_class

,scale,plaque,pustule,patch,papule,dermatoglyph_disruption,open_comedo,diagnosis_acne,diagnosis_actinic_keratosis,diagnosis_psoriasis,diagnosis_seborrheic_dermatitis,diagnosis_viral_warts,diagnosis_vitiligo,area_acral_distribution,area_exposed_areas,area_extensor_sites,area_seborrheic_region
0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1
1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
2,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
3,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1
4,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
260,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
261,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
262,1,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0


## Measure scores

* By only supplying targets and predictions, f1 scores are calculated for diagnosis_variables, area_variables and characteristics_variables. These are returned as single values in a dictionary
* If 'type' is supplied (f1:default/recall/precission), said method will be used for measurements
* If 'average' is supplied (macro:default/samples/micro), said average filter will be applied to measurement method
* If 'single_char_scores' is set to True, individual characteristic scores will be calculated with default=f1 or supplied score type
* If 'single_area_scores' is set to True, individual area scores will be calculated with default=f1 or supplied score type

In [4]:
f1_score(y_true=ground_truths['scale'], y_pred=pred_class['scale'], average='macro', zero_division=0)

0.49255952380952384

In [5]:
print('Metric Scores overall:')
print(tools.compute_metrics_scores(ground_truths, pred_class))
print('')
print('Metric Scores wit recall and micro:')
print(tools.compute_metrics_scores(ground_truths, pred_class, type=recall_score, average='micro'))
print('')
print('Metric Scores with single characteristics:')
print(tools.compute_metrics_scores(ground_truths, pred_class, single_char_scores=True))
print('')
print('Metric Scores with single characteristics and single area:')
print(tools.compute_metrics_scores(ground_truths, pred_class, single_char_scores=True, single_area_scores=True))

Metric Scores overall:
{'characteristics': 0.38218723386790615, 'diagnosis': 0.1781317299385846, 'area': 0.3255980700561981}

Metric Scores wit recall and micro:
{'characteristics': 0.5837837837837838, 'diagnosis': 0.1893939393939394, 'area': 0.38257575757575757}

Metric Scores with single characteristics:
{'characteristics': {'total': 0.38218723386790615, 'singles': {'scale': 0.630952380952381, 'plaque': 0.776470588235294, 'pustule': 0.061538461538461535, 'patch': 0.5396825396825398, 'papule': 0.5866666666666667, 'dermatoglyph_disruption': 0.0, 'open_comedo': 0.08000000000000002}}, 'diagnosis': 0.1781317299385846, 'area': 0.3255980700561981}

Metric Scores with single characteristics and single area:
{'characteristics': {'total': 0.38218723386790615, 'singles': {'scale': 0.630952380952381, 'plaque': 0.776470588235294, 'pustule': 0.061538461538461535, 'patch': 0.5396825396825398, 'papule': 0.5866666666666667, 'dermatoglyph_disruption': 0.0, 'open_comedo': 0.08000000000000002}}, 'diagno

In [7]:
stats = tools.compute_metrics_scores(ground_truths, pred_class, single_char_scores=True, single_area_scores=True)

print(stats['characteristics']['total'])

print(np.array(list(stats['characteristics']['singles'].values())).mean())

0.38218723386790615
0.48637346945407484


## Test of variable f1 scores

In [8]:
diagnosis_variables, area_variables, characteristics_variables = tools.getVariableGroups()

thresholds = np.arange(0, 1, 0.01)
def predict(y_est_prob, threshold):
	return (y_est_prob >= threshold).astype('int')
    
# Basic characteristics
basic_chars_thresholds = []
basic_chars_scores = []
for char in characteristics_variables:
    scores = [f1_score(ground_truths[char], predict(predictions[char], t)) for t in thresholds]
    i_opt = np.argmax(scores)
    basic_chars_thresholds.append(thresholds[i_opt])
    basic_chars_scores.append(scores[i_opt])
thresholds_scores_opt = pd.DataFrame(
    [basic_chars_thresholds, basic_chars_scores],
    index=['threshold','score'],
    columns=characteristics_variables)

thresholds_final = thresholds_scores_opt.loc['threshold',:]
thresholds_scores_opt

,scale,plaque,pustule,patch,papule,dermatoglyph_disruption,open_comedo
threshold,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000
score,0.727711,0.818792,0.339623,0.649616,0.693069,0.17301,0.37037


In [9]:
# General issue with f1
gt1 = ground_truths.iloc[:,0]
pred1 = predictions.iloc[:,0]
idx = np.argmax(scores)
print('Best Threshold=%f, F-Score=%.3f' % (thresholds[idx], scores[idx]))
for i in range(0,len(scores)):
    print(f'Threshold={thresholds[i]:.2f}, f1-Score={scores[i]:.3f}')

Best Threshold=0.000000, F-Score=0.370
Threshold=0.00, f1-Score=0.370
Threshold=0.01, f1-Score=0.080
Threshold=0.02, f1-Score=0.080
Threshold=0.03, f1-Score=0.080
Threshold=0.04, f1-Score=0.080
Threshold=0.05, f1-Score=0.080
Threshold=0.06, f1-Score=0.080
Threshold=0.07, f1-Score=0.080
Threshold=0.08, f1-Score=0.080
Threshold=0.09, f1-Score=0.080
Threshold=0.10, f1-Score=0.080
Threshold=0.11, f1-Score=0.080
Threshold=0.12, f1-Score=0.080
Threshold=0.13, f1-Score=0.080
Threshold=0.14, f1-Score=0.080
Threshold=0.15, f1-Score=0.080
Threshold=0.16, f1-Score=0.080
Threshold=0.17, f1-Score=0.080
Threshold=0.18, f1-Score=0.080
Threshold=0.19, f1-Score=0.080
Threshold=0.20, f1-Score=0.080
Threshold=0.21, f1-Score=0.080
Threshold=0.22, f1-Score=0.080
Threshold=0.23, f1-Score=0.080
Threshold=0.24, f1-Score=0.080
Threshold=0.25, f1-Score=0.080
Threshold=0.26, f1-Score=0.080
Threshold=0.27, f1-Score=0.080
Threshold=0.28, f1-Score=0.080
Threshold=0.29, f1-Score=0.080
Threshold=0.30, f1-Score=0.080
